# Tables

In [1]:
import os
import warnings
import pandas as pd
from tableone import TableOne

In [2]:
# Import dataset
df = pd.read_excel("data/dataset_fda_devices.xlsx")
df["Recall"] = df["Recall"].replace({"no": "Available", "yes": "Recalled"})

In [3]:
df["Summary Malfunction Reporting"] = df["Summary Malfunction Reporting"].replace({"no": "ineligible"}).apply(str.capitalize)
df["Clinical Performance Study"] = df["Clinical Performance Study"].apply(lambda x: "Missing information" if x not in ["yes", "no"] else x.capitalize())
for col in ["DICOM compliance", "CP results sex available", "CP results age available"]:
    df[col] = df[col].apply(lambda x: "Missing information" if x != "yes" else "Yes")

In [4]:
# Dates to datetime format
date_columns = [col for col in df.columns if "Date" in col]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df["Decision Time"] = (df["Decision Date"] - df["Date Received"]).dt.days
df["Recall Time"] = (df["FDA Recall Posting Date"] - df["Date of Final Decision"]).dt.days

In [5]:
panels = ["Anesthesiology", "Cardiovascular", "Gastroenterology/Urology", "Hematology", "Neurology", "Ophthalmic", "Radiology"]
df["Panel"] = df["Panel (Lead)"].apply(lambda x: "Other" if x not in panels else x)

In [6]:
warnings.filterwarnings('ignore')
columns = ["Geographic area", "Panel", "Submission Type", "Eligible for Third Party Review", "Summary Malfunction Reporting",
           "Implanted Device", "Physical State", "DICOM compliance", "Clinical Performance Study", "Decision Time", "Recall Time"]
non_normal = ["Decision Time", "Recall Time"]
categorical = [col for col in columns if col not in non_normal]
for column in columns:
    if df[column].dtype =='0':
        df[column] = df[column].apply(str.capitalize)
table_all = TableOne(df, columns=columns, categorical=categorical, nonnormal=non_normal, groupby="Recall", missing=False, decimals=0)
os.makedirs("tables", exist_ok=True)
table_all.to_excel(f"tables/Table1_1.xlsx")
print(table_all.tabulate(tablefmt="fancygrid"))

                                                                  Overall         Available      Recalled
--------------------------------------  ------------------------  --------------  -------------  --------------
n                                                                 903             860            43
Geographic area, n (%)                  Asia                      154 (17)        147 (17)       7 (16)
                                        Europe                    183 (20)        175 (20)       8 (19)
                                        Northamerica              467 (52)        442 (51)       25 (58)
                                        other                     99 (11)         96 (11)        3 (7)
Panel, n (%)                            Anesthesiology            9 (1)           9 (1)
                                        Cardiovascular            91 (10)         88 (10)        3 (7)
                                        Gastroenterology/Urology  14 (2)   

In [7]:
df_cp = df[df["Clinical Performance Study"] == "Yes"].copy()
columns = ["CP Study-type", "CP results sex available", "CP results age available"]
table_all = TableOne(df_cp, columns=columns, categorical=columns, groupby="Recall", missing=False, decimals=0)
os.makedirs("tables", exist_ok=True)
table_all.to_excel(f"tables/Table1_2.xlsx")
print(table_all.tabulate(tablefmt="fancygrid"))

                                                      Overall    Available    Recalled
-------------------------------  -------------------  ---------  -----------  ----------
n                                                     505        492          13
CP Study-type, n (%)             Missing              259 (51)   251 (51)     8 (62)
                                 Prospective          41 (8)     38 (8)       3 (23)
                                 Randomized           12 (2)     11 (2)       1 (8)
                                 Retrospective        193 (38)   192 (39)     1 (8)
CP results sex available, n (%)  Missing information  360 (71)   348 (71)     12 (92)
                                 Yes                  145 (29)   144 (29)     1 (8)
CP results age available, n (%)  Missing information  388 (77)   377 (77)     11 (85)
                                 Yes                  117 (23)   115 (23)     2 (15)
